# K-nächste Nachbarn (K-nearest Neighbors)
Wir wenden den K-nächste-Nachbarn-Algorithmus auf den IRIS-Datensatz an.

In [ ]:
# Wir laden den Datensatz
import pandas as pd
url = "https://raw.githubusercontent.com/troescherw/datasets/master/iris.csv"
iris = pd.read_csv(url, delimiter=";")
iris



In [ ]:
# Aufteilen in X und y
X = iris.iloc[:, :4]
y = iris.SpeciesID

In [ ]:
# Aufteilen in Trainings- und Testdaten
# "Training" bedeutet hier das Erstellen einer Abstandsmatrix

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.3)

Wir erstellen das Modell und verwenden für k den Wert 5.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5).fit(X_train, y_train)


Wir klassifizieren die Objekte aus dem Test-Datensatz.

In [ ]:
pred = knn.predict(X_test)
pred

Wir stellen das Ergebnis in einer Confusion Matrix dar und berechnen die Accuracy.

In [ ]:
from sklearn.metrics import plot_confusion_matrix
import numpy as np

classes = ["Setosa", "Versicolor", "Virginica"]
_=plot_confusion_matrix(knn, X_test, y_test, display_labels=classes)



In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred)

## Optimierung des Algorithmus

Für die Zuordnung des Objektes zu einer Klasse gibt es mehrere Möglichkeiten: Im einfachsten Fall wird wie schon erwähnt eine schlichte Mehrheitsentscheidung getroffen, was aber nicht immer das beste Ergebnis liefert, insbesondere wenn sich ähnlich oder sogar gleich viele Objekte der jeweiligen Klasse in unmittelbarer Nachbarschaft befinden. Daher kann man altnernativ die Objekte in der Nachbarschaft gewichten: Je näher ein Objekt, desto mehr "Gewicht" erhält die jeweilige Klasse.

Dem Konstruktor der Klasse *KNeighborsClassifier* kann man deshalb noch das Attribut *weights* bestimmen. Standardmäßig ist dies auf *uniform* gesetzt, was der ersten, einfachen Methode entspricht. Man kann es aber auch auf den Wert *distance* setzen, dann werden die Abstände gewichtet.

Im folgenden Skript wollen wir nun anhand eines Datensatzes das optimale K und auch die optimale Gewichtung der Abstände bestimmen. Wir verwenden hierfür einen Datensatz aus dem *sklearn*-Package, der über chemische Analysedaten von Weinen verfügt. Jeder der 178 Weine stammt von einem von drei italienischen Winzern ("cultivator"). Wie gut kann unser Modell vorhersagen, von welchem Winzer der Wein stammt?

Um unser Modell zu optimieren werden wir ein K von 1 bis 10 verwenden und für jedes K ein unterschiedliches Verfahren für die Gewichtung verwenden. Es werden also insgesamt 20 Modelle durchprobiert und für jedes ermitteln wir die Accuracy. Die Kombination aus K und dem Typ für die Gewichtung mit der höchsten Accuracy liefert (vermutlich) die besten Hyperparameter.

In [ ]:
# Laden des Datensatzes
from sklearn.datasets import load_wine

wines = load_wine()
print(wines.DESCR)

In [ ]:
# Aufteilen in Trainings- und Testdaten
# "Training" bedeutet hier das Erstellen einer Abstandsmatrix

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(wines.data, wines.target, shuffle=True, test_size=0.3, random_state=23 )

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np


weights = ["uniform", "distance"]
ks = np.arange(1,11)

   
for weight in weights:
    for k in ks:
        knn = KNeighborsClassifier(n_neighbors=k, weights=weight).fit(X_train, y_train)
        print(f"K={k}, weights={weight}, Accuracy = {knn.score(X_test, y_test):.3}")
        

## KNN für Regressionsanalyse
KNN kann auch für Regression verwendet werden. Im Beispiel werden Mietpreise für Wohnungen vorherhergesagt.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
import numpy as np

# Mietpreise
X = np.array([
        [69, 1685],
        [28, 625],
        [42, 524],
        [113, 2100],
        [54, 1200],
        [43, 750],
        [62, 1178],
        [24, 900],
        [33, 715],
        [92, 2915],
        [53, 1440]
        ]
    )

knn = KNeighborsRegressor(n_neighbors=3).fit(X[:,0].reshape(-1,1), X[:,1].reshape(-1,1))

# Predictions
qm = np.array([50,70,90]).reshape(-1,1)
print(knn.predict(qm))

In [ ]:
# Visualisierung
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")
plt.scatter(X[:,0], X[:,1], label="Trainingsdaten")
plt.scatter(qm, knn.predict(qm), label="Vorhersagen")
plt.legend()
plt.title("Mietpreise Wohnungen mit KNN")
plt.xlabel(r"Größe der Wohnung in $m^2$")
plt.ylabel("Mietpreis in €")
plt.show()